# Test 6

## Objective
- The goal of this test is to have an idea of the existing tools to implement the provenance in a program of CTA using the ctapipe library

## Context
- This test is done with the muon_reconstruction.py program 
- This program uses ctapipe and provenance modules/libraries
- The Provenance database is defined in memory and accessed with the sqlalchemy library

## Structure of the notebook
- Definition the Provenance database structure
- Definition of the muon_reconstruction tool
- Addition of the muon_reconstruction (named ctapipe_display_muons) activity description in the provenance database
- Execution of the muon_reconstruction program
- Addition of the provenance information of the job in the database
- Query the provenance database and store the result in a file

## Remarks
- Activity.activityDescription = concat(activity_name, '_', ctapipe_version)
- Entity.id = hash(file)
- Used.id, WasGeneratedBy.id, WasAttributedTo.id, WasAssociatedWith.id are interger and autoincremented
- A lot of empty fields and problem to associate Entity with EntityDescription
- Dates are curreuntly stored as strings
- Inheritance is implemented as jouned table inheritance (dependant tables) => addition of the classType attribute in the Entity and EntityDescription classes

## Definition of the Provenance database structure

In [1]:
# Connect to the database (here in memory)
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

# Import sqlachemy modules to create objects mapped with tables
from sqlalchemy import Table, Column, ForeignKey
from sqlalchemy import Integer, String
from sqlalchemy import exists
from sqlalchemy.orm import relationship

# Declare a declarative_base to map objets and tables
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()


## Remarks:
- Need to add theclassType attribute to distinguish the inherited classes


In [2]:
# Define the Activity class mapped to the activities table
class Activity(Base):
    __tablename__ = 'activities'
    ordered_attribute_list = ['id','name','startTime','endTime','comment','activityDescription_id']
    id        = Column(String, primary_key=True)
    name      = Column(String)
    startTime = Column(String) 
    endTime   = Column(String)
    comment   = Column(String) 
    activityDescription_id = Column(String, ForeignKey("activityDescriptions.id"))
    activityDescription    = relationship("ActivityDescription")
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "Activity.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    
# Define the Entity class mapped to the entities table
class Entity(Base):
    __tablename__ = 'entities'
    ordered_attribute_list = ['id','classType','name','location','generatedAtTime','invalidatedAtTime','comment','entityDescription_id' ]
    id                  = Column(String, primary_key=True)
    name                = Column(String)
    location            = Column(String) 
    generatedAtTime     = Column(String)
    invalidatedAtTime   = Column(String)
    comment             = Column(String)
    classType           = Column(String)
    entityDescription_id   = Column(String, ForeignKey("entityDescriptions.id")) 
    entityDescription      = relationship("EntityDescription")
    __mapper_args__ = {
        'polymorphic_identity':'entity',
        'polymorphic_on': classType
    }
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "Entity.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    

In [3]:
# Define the Used class mapped to the used table
class Used(Base):
    __tablename__ = 'used'
    ordered_attribute_list = ['id','role','time','activity_id','entity_id']
    id = Column(Integer, primary_key=True, autoincrement=True) 
    role     = Column(String, nullable=True)
    time     = Column(String)
    activity_id = Column(String, ForeignKey('activities.id'))
    activity = relationship("Activity")
    entity_id = Column(String, ForeignKey('entities.id'))
    entity = relationship("Entity")
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "Used.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    
# Define the WasGeneratedBy class mapped to the wasGeneratedBy table
class WasGeneratedBy(Base):
    __tablename__ = 'wasGeneratedBy'
    ordered_attribute_list = ['id','role','activity_id','entity_id']
    id = Column(Integer, primary_key=True, autoincrement=True)
    role     = Column(String, nullable=True)
    activity_id = Column(String, ForeignKey('activities.id'))
    activity = relationship("Activity")
    entity_id = Column(String, ForeignKey('entities.id'))
    entity = relationship("Entity")
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "WasGeneratedBy.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response

## Remarks
the value attribute is not the best name (keyword in some languages)

In [4]:
# Define the ValueEntity class mapped to the valueEntities table
class ValueEntity(Entity):
    __tablename__ = 'valueEntities'
    #ordered_attribute_list = Entity.ordered_attribute_list+['value']
    ordered_attribute_list = Entity.ordered_attribute_list
    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    valueXX = Column(String)
    __mapper_args__ = {'polymorphic_identity':'value'}
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "ValueEntity.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    
# Define the DatasetEntity class mapped to the datasetEntities table
class DatasetEntity(Entity):
    __tablename__ = 'datasetEntities'
    ordered_attribute_list = Entity.ordered_attribute_list
    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    __mapper_args__ = {'polymorphic_identity':'dataset'}
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "DatasetEntity.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    
# Define the Parameter class mapped to the parameters table
class Parameter(ValueEntity):
    __tablename__ = 'parameters'
    #ordered_attribute_list = Entity.ordered_attribute_list + ['valueType', 'unit', 'ucd', 'utype']
    ordered_attribute_list = Entity.ordered_attribute_list
    id = Column(String, ForeignKey('valueEntities.id'), primary_key=True)
    valueType = Column(String)
    unit      = Column(String)
    ucd       = Column(String)
    utype     = Column(String)
    __mapper_args__ = {'polymorphic_identity':'parameter'}
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "Parameter.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response

In [5]:
# Define the Agent class mapped to the agents table
class Agent(Base):
    __tablename__ = 'agents'
    ordered_attribute_list = ['id','name','type','email','affiliation','phone','address']
    id                  = Column(String, primary_key=True)
    name                = Column(String)
    type                = Column(String) 
    email               = Column(String)
    affiliation         = Column(String)
    phone               = Column(String)
    address             = Column(String)
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "Agent.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    
# Define the WasAssociatedWith class mapped to the wasAssociatedWith table
class WasAssociatedWith(Base):
    __tablename__ = 'wasAssociatedWith'
    ordered_attribute_list = ['id','activity','agent','role']
    id       = Column(Integer, primary_key=True, autoincrement=True)
    activity = Column(String, ForeignKey("activities.id")) 
    agent    = Column(String, ForeignKey("agents.id")) 
    role     = Column(String, nullable=True)
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "WasAssociatedWith.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    
# Define the WasAttributedTo class mapped to the wasAttributedTo table
class WasAttributedTo(Base):
    __tablename__ = 'wasAttributedTo'
    ordered_attribute_list = ['id','entity','agent','role']
    id       = Column(Integer, primary_key=True, autoincrement=True)
    entity   = Column(String, ForeignKey("entities.id")) 
    agent    = Column(String, ForeignKey("agents.id")) 
    role     = Column(String, nullable=True)
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "WasAttributedTo.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response

In [6]:
# Define the ActivityDescription class mapped to the activityDescriptions table
class ActivityDescription(Base):
    __tablename__ = 'activityDescriptions'
    ordered_attribute_list = ['id','name','activity_type','activity_subtype','version','doculink']
    id                 = Column(String, primary_key=True)
    name               = Column(String)
    activity_type      = Column(String) 
    activity_subtype   = Column(String)
    version            = Column(String)
    doculink           = Column(String)
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "ActivityDescription.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response

# Define the EntityDescription class mapped to the entityDescriptions table
class EntityDescription(Base):
    __tablename__ = 'entityDescriptions'
    ordered_attribute_list = ['id','name','type','description','doculink','classType']
    id                 = Column(String, primary_key=True)
    name               = Column(String)
    type               = Column(String)
    description        = Column(String)
    doculink           = Column(String)
    classType         = Column(String)
    __mapper_args__ = {
        'polymorphic_identity':'entityDescription',
        'polymorphic_on':classType
    }
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "EntityDescription.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response

In [7]:
# Define the UsageDescription class mapped to the usageDescriptions table
class UsageDescription(Base):
    __tablename__ = 'usageDescriptions'
    ordered_attribute_list = ['id','role','description','type','activityDescription_id','entityDescription_id']
    id = Column(String, primary_key=True) 
    role        = Column(String, nullable=True)
    description = Column(String)
    type        = Column(String)  
    activityDescription_id = Column(String, ForeignKey('activityDescriptions.id'))
    activityDescription = relationship("ActivityDescription")
    entityDescription_id = Column(String, ForeignKey('entityDescriptions.id'))
    entityDescription = relationship("EntityDescription")
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "UsageDescription.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    
# Define the GenerationDescription class mapped to the generationDescriptions table
class GenerationDescription(Base):
    __tablename__ = 'generationDescriptions'
    ordered_attribute_list = ['id','role','description','type','activityDescription_id','entityDescription_id']
    id          = Column(String, primary_key=True)
    role        = Column(String, nullable=True)
    description = Column(String)
    type        = Column(String)
    # Relations
    activityDescription_id = Column(String, ForeignKey('activityDescriptions.id'))
    activityDescription = relationship("ActivityDescription")
    entityDescription_id = Column(String, ForeignKey('entityDescriptions.id'))
    entityDescription = relationship("EntityDescription")
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "GenerationDescription.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    

In [8]:
# Define the ValueDescription class mapped to the valueDescriptions table
class ValueDescription(EntityDescription):
    __tablename__ = 'valueDescriptions'
    #ordered_attribute_list = EntityDescription.ordered_attribute_list + ['valueType','unit','ucd','utype']
    ordered_attribute_list = EntityDescription.ordered_attribute_list
    id = Column(String, ForeignKey('entityDescriptions.id'), primary_key=True)
    valueType = Column(String)
    unit      = Column(String)
    ucd       = Column(String)
    utype     = Column(String)
    __mapper_args__ = {'polymorphic_identity':'valueDescription'}
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "ValueDescription.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    
# Define the DatasetDescription class mapped to the datasetDescriptions table
class DatasetDescription(EntityDescription):
    __tablename__ = 'datasetDescriptions'
    #ordered_attribute_list = EntityDescription.ordered_attribute_list + ['contentType']
    ordered_attribute_list = EntityDescription.ordered_attribute_list
    id = Column(String, ForeignKey('entityDescriptions.id'), primary_key=True)
    contentType = Column(String)
    __mapper_args__ = {'polymorphic_identity':'datasetDescription'}
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "DatasetDescription.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response
    
# Define the ParameterDescription class mapped to the parameterDescriptions table
class ParameterDescription(ValueDescription):
    __tablename__ = 'parameterDescriptions'
    #ordered_attribute_list = ValueDescription.ordered_attribute_list + ['min','max','options','default']
    ordered_attribute_list = EntityDescription.ordered_attribute_list
    id = Column(String, ForeignKey('valueDescriptions.id'), primary_key=True)
    min     = Column(String)
    max     = Column(String)
    options = Column(String)
    default = Column(String)
    __mapper_args__ = {'polymorphic_identity':'parameterDescription'}
    def __repr__(self):
        response = ""
        for attribute in self.ordered_attribute_list:
            response += "ParameterDescription.%s=%s\n" %(attribute,self.__dict__[attribute])
        return response

In [9]:
# sqlalchemy creates the database for me
Base.metadata.create_all(engine)

2019-04-02 09:40:35,739 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-04-02 09:40:35,741 INFO sqlalchemy.engine.base.Engine ()
2019-04-02 09:40:35,743 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-04-02 09:40:35,744 INFO sqlalchemy.engine.base.Engine ()
2019-04-02 09:40:35,745 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("activities")
2019-04-02 09:40:35,746 INFO sqlalchemy.engine.base.Engine ()
2019-04-02 09:40:35,748 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("entities")
2019-04-02 09:40:35,749 INFO sqlalchemy.engine.base.Engine ()
2019-04-02 09:40:35,752 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("used")
2019-04-02 09:40:35,754 INFO sqlalchemy.engine.base.Engine ()
2019-04-02 09:40:35,757 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("wasGeneratedBy")
2019-04-02 09:40:35,758 INFO sqlalchemy.engine.base.Engine ()
2019-04-02 09:40:35,760 INFO s

2019-04-02 09:40:35,870 INFO sqlalchemy.engine.base.Engine ()
2019-04-02 09:40:35,871 INFO sqlalchemy.engine.base.Engine COMMIT
2019-04-02 09:40:35,872 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "wasAssociatedWith" (
	id INTEGER NOT NULL, 
	activity VARCHAR, 
	agent VARCHAR, 
	role VARCHAR, 
	PRIMARY KEY (id), 
	FOREIGN KEY(activity) REFERENCES activities (id), 
	FOREIGN KEY(agent) REFERENCES agents (id)
)


2019-04-02 09:40:35,873 INFO sqlalchemy.engine.base.Engine ()
2019-04-02 09:40:35,874 INFO sqlalchemy.engine.base.Engine COMMIT
2019-04-02 09:40:35,875 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "wasAttributedTo" (
	id INTEGER NOT NULL, 
	entity VARCHAR, 
	agent VARCHAR, 
	role VARCHAR, 
	PRIMARY KEY (id), 
	FOREIGN KEY(entity) REFERENCES entities (id), 
	FOREIGN KEY(agent) REFERENCES agents (id)
)


2019-04-02 09:40:35,876 INFO sqlalchemy.engine.base.Engine ()
2019-04-02 09:40:35,877 INFO sqlalchemy.engine.base.Engine COMMIT
2019-04-02 09:40:35,878 INFO sqlalchemy.engi

## muon_reconstruction definition

In [10]:
"""
Example to load raw data (hessio format), calibrate and reconstruct muon
ring parameters, and write the muon ring and intensity parameters to an output
table.

The resulting output can be read e.g. using `pandas.read_hdf(filename,
'muons/LSTCam')`
"""

import warnings
from collections import defaultdict

from tqdm import tqdm

from ctapipe.calib import CameraCalibrator
from ctapipe.core import Provenance
from ctapipe.core import Tool, ToolConfigurationError
from ctapipe.core import traits as t
from ctapipe.image.muon.muon_diagnostic_plots import plot_muon_event
from ctapipe.image.muon.muon_reco_functions import analyze_muon_event
from ctapipe.io import EventSourceFactory
from ctapipe.io import HDF5TableWriter

warnings.filterwarnings("ignore")  # Supresses iminuit warnings


def _exclude_some_columns(subarray, writer):
    """ a hack to exclude some columns of all output tables here we exclude
    the prediction and mask quantities, since they are arrays and thus not
    readable by pandas.  Also, prediction currently is a variable-length
    quantity (need to change it to be fixed-length), so it cannot be written
    to a fixed-length table.
    """
    all_camids = {str(x.camera) for x in subarray.tel.values()}
    for cam in all_camids:
        writer.exclude(cam, 'prediction')
        writer.exclude(cam, 'mask')

class MuonDisplayerTool(Tool):
    name = 'ctapipe-display-muons'
    description = t.Unicode(__doc__)

    events = t.Unicode("",
                       help="input event data file").tag(config=True)

    outfile = t.Unicode("muons.hdf5", help='HDF5 output file name').tag(
        config=True)

    display = t.Bool(
        help='display the camera events', default=False
    ).tag(config=True)

    classes = t.List([
        CameraCalibrator, EventSourceFactory
    ])

    aliases = t.Dict({
        'input': 'MuonDisplayerTool.events',
        'outfile': 'MuonDisplayerTool.outfile',
        'display': 'MuonDisplayerTool.display',
        'max_events': 'EventSourceFactory.max_events',
        'allowed_tels': 'EventSourceFactory.allowed_tels',
    })

    def setup(self):
        if self.events == '':
            raise ToolConfigurationError("please specify --input <events file>")
        self.log.debug("input: %s", self.events)
        self.source = EventSourceFactory.produce(input_url=self.events)
        self.calib = CameraCalibrator(
            config=self.config, tool=self, eventsource=self.source
        )
        self.writer = HDF5TableWriter(self.outfile, "muons")

    def start(self):

        numev = 0
        self.num_muons_found = defaultdict(int)

        for event in tqdm(self.source, desc='detecting muons'):

            self.calib.calibrate(event)
            muon_evt = analyze_muon_event(event)

            if numev == 0:
                _exclude_some_columns(event.inst.subarray, self.writer)

            numev += 1

            if not muon_evt['MuonIntensityParams']:
                # No telescopes  contained a good muon
                continue
            else:
                if self.display:
                    plot_muon_event(event, muon_evt)

                for tel_id in muon_evt['TelIds']:
                    idx = muon_evt['TelIds'].index(tel_id)
                    intens_params = muon_evt['MuonIntensityParams'][idx]

                    if intens_params is not None:
                        ring_params = muon_evt['MuonRingParams'][idx]
                        cam_id = str(event.inst.subarray.tel[tel_id].camera)
                        self.num_muons_found[cam_id] += 1
                        self.log.debug("INTENSITY: %s", intens_params)
                        self.log.debug("RING: %s", ring_params)
                        self.writer.write(table_name=cam_id,
                                          containers=[intens_params,
                                                      ring_params])

                self.log.info(
                    "Event Number: %d, found %s muons",
                    numev, dict(self.num_muons_found)
                )

    def finish(self):
        Provenance().add_output_file(self.outfile,
                                     role='dl1.tel.evt.muon')
        self.writer.close()





## Descriptions added in the Provenance database

In [11]:
# Define the session to talk to the database
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

# Create an instance of ActivityDescription 
actDesc1 = ActivityDescription(id='ctapipe_display_muons_0.6.1',name='ctapipe_display_muons',\
                               activity_type='',activity_subtype='',version='0.6.1', doculink='')

# Create the description of input entities
dataDesc1 = DatasetDescription(id='proton_events', name='protons', description='proton file', classType='datasetDescription')
usedDesc1 = UsageDescription(id='ctapipe_display_muons_0.6.1_proton_events',activityDescription=actDesc1, entityDescription=dataDesc1, role="dl0.sub.evt")
# Create the description of output entities
dataDesc2  = DatasetDescription(id='muons_hdf5', name='muons', description='muon file', classType='datasetDescription')
wGBDesc1   = GenerationDescription(id='ctapipe_display_muons_0.6.1_muons_hdf5',activityDescription=actDesc1, entityDescription=dataDesc2, role="dl0.sub.evt")
valueDesc1 = ValueDescription(id='status', classType='valueDescription')
wGBDesc2   = GenerationDescription(id='ctapipe_display_muons_0.6.1_status', activityDescription=actDesc1, entityDescription=valueDesc1, role="quality")

# Put the instance in the database
session.add(actDesc1)
session.add(dataDesc1)
session.add(usedDesc1)
session.add(dataDesc2)
session.add(wGBDesc1)
session.add(valueDesc1)
session.add(wGBDesc2)
session.commit()

2019-04-02 09:40:41,964 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-04-02 09:40:41,968 INFO sqlalchemy.engine.base.Engine INSERT INTO "activityDescriptions" (id, name, activity_type, activity_subtype, version, doculink) VALUES (?, ?, ?, ?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO "activityDescriptions" (id, name, activity_type, activity_subtype, version, doculink) VALUES (?, ?, ?, ?, ?, ?)


2019-04-02 09:40:41,970 INFO sqlalchemy.engine.base.Engine ('ctapipe_display_muons_0.6.1', 'ctapipe_display_muons', '', '', '0.6.1', '')


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO "generationDescriptions" (id, role, description, type, "activityDescription_id", "entityDescription_id") VALUES (?, ?, ?, ?, ?, ?)


2019-04-02 09:40:42,006 INFO sqlalchemy.engine.base.Engine (('ctapipe_display_muons_0.6.1_muons_hdf5', 'dl0.sub.evt', None, None, 'ctapipe_display_muons_0.6.1', 'muons_hdf5'), ('ctapipe_display_muons_0.6.1_status', 'quality', None, None, 'ctapipe_display_muons_0.6.1', 'status'))


INFO:sqlalchemy.engine.base.Engine:(('ctapipe_display_muons_0.6.1_muons_hdf5', 'dl0.sub.evt', None, None, 'ctapipe_display_muons_0.6.1', 'muons_hdf5'), ('ctapipe_display_muons_0.6.1_status', 'quality', None, None, 'ctapipe_display_muons_0.6.1', 'status'))


2019-04-02 09:40:42,011 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


## muon_reconstruction job

In [12]:
from ctapipe.core import Provenance
from pprint import pprint
p = Provenance()  # note this is a singleton, so only ever one global provenence object
p.clear()

p.start_activity()
    
myTool = MuonDisplayerTool()
myTool.run(['--input=/Users/bourgeat/Documents/CTA/Provenance/tests/proton_20deg_180deg_run22___cta-prod3-demo-2147m-LaPalma-baseline.simtel.gz'])

myTool = MuonDisplayerTool()
myTool.run(['--input=/Users/bourgeat/Documents/CTA/Provenance/tests/proton_20deg_180deg_run22___cta-prod3-demo-2147m-LaPalma-baseline.simtel.gz'])

p.finish_activity()

INFO [MuonDisplayerTool] (tool/initialize): ctapipe version 0.6.1
INFO [MuonDisplayerTool] (tool/run): Starting: ctapipe-display-muons
INFO [MuonDisplayerTool.ChargeExtractorFactory] (factory/_product): Obtaining NeighbourPeakIntegrator from ChargeExtractorFactory
INFO [MuonDisplayerTool.WaveformCleanerFactory] (factory/_product): Obtaining NullWaveformCleaner from WaveformCleanerFactory
INFO [MuonDisplayerTool.CameraR1CalibratorFactory] (factory/_product): Obtaining HESSIOR1Calibrator from CameraR1CalibratorFactory
INFO [MuonDisplayerTool.CameraDL0Reducer] (dl0/__init__): Applying no data volume reduction in the conversion from R1 to DL0
detecting muons: 8it [00:00,  8.38it/s]
INFO [MuonDisplayerTool] (tool/run): Finished: ctapipe-display-muons
INFO [MuonDisplayerTool] (tool/run): Output: /Users/bourgeat/Documents/CTA/Provenance/ctapipe/tests/muons.hdf5
INFO [MuonDisplayerTool] (tool/initialize): ctapipe version 0.6.1
INFO [MuonDisplayerTool] (tool/run): Starting: ctapipe-display-muon

In [13]:
p.finished_activity_names

['ctapipe-display-muons',
 'ctapipe-display-muons',
 '/Users/bourgeat/anaconda3/bin/python']

In [14]:
p.provenance[:-1]

[{'activity_name': 'ctapipe-display-muons',
  'activity_uuid': 'ed7e27d4-a0a7-43a2-97a1-511348dd37bd',
  'config': {'MuonDisplayerTool': {'events': '/Users/bourgeat/Documents/CTA/Provenance/tests/proton_20deg_180deg_run22___cta-prod3-demo-2147m-LaPalma-baseline.simtel.gz'}},
  'duration_min': 0.016816666666636948,
  'input': [{'role': 'dl0.sub.evt',
    'url': '/Users/bourgeat/Documents/CTA/Provenance/tests/proton_20deg_180deg_run22___cta-prod3-demo-2147m-LaPalma-baseline.simtel.gz'}],
  'output': [{'role': 'dl1.tel.evt.muon',
    'url': '/Users/bourgeat/Documents/CTA/Provenance/ctapipe/tests/muons.hdf5'}],
  'start': {'time_utc': '2019-04-02T07:40:47.663'},
  'status': 'completed',
  'stop': {'time_utc': '2019-04-02T07:40:48.672'},
  'system': {'arguments': ['/Users/bourgeat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py',
    '-f',
    '/Users/bourgeat/Library/Jupyter/runtime/kernel-19cd8f1b-c4ea-4fbc-87d8-a0e25980ce43.json'],
   'ctapipe_resources_version': '0.2.15',
  

## Provenance database update

In [15]:
import hashlib, uuid, os
BLOCKSIZE = 65536
hasher = hashlib.sha1()

def get_file_id(url):
    '''
    # Computation of the hash of the file to determine the id of it
    with open(cta_input['url'], 'rb') as afile:
        buf = afile.read(BLOCKSIZE)
        while len(buf) > 0:
            hasher.update(buf)
            buf = afile.read(BLOCKSIZE)
            return hasher.hexdigest()
    '''
    logical_name = url.split('/')[-1] 
    name = logical_name + str(os.path.getctime(url))
    file_uuid = uuid.uuid5(uuid.NAMESPACE_URL, name)
    if session.query(exists().where(DatasetEntity.id==file_uuid)):
        print ("cle existante")
        return str(file_uuid)
    else:
        print ("cle non existante")
        return ""

    # universal unique id
    #return uuid.uuid4()

def set_file_id(url):
    logical_name = url.split('/')[-1] 
    name = logical_name + str(os.path.getctime(url))
    file_uuid = uuid.uuid5(uuid.NAMESPACE_URL, name)
    return str(file_uuid)

def add_activity(session, cta_activity):
    if not session.query(exists().where(Activity.id==cta_activity['activity_uuid'])): # for the tests
        current_activity = Activity(id=cta_activity['activity_uuid'])
        current_activity.name=cta_activity['activity_name']
        current_activity.startTime=cta_activity['start']['time_utc']
        current_activity.endTime=cta_activity['stop']['time_utc']
        current_activity.comment=''
        current_activity.activityDescription_id=cta_activity['activity_name']+'_'+cta_activity['system']['ctapipe_version']
        session.add(current_activity)
    
        # Association with the agent
        wAW = WasAssociatedWith()
        wAW.activity = cta_activity['activity_uuid']
        wAW.agent    = "CTAO"
        #wAW.role = ?
        session.add(wAW)
    

# CTAO Agent
agent = Agent(id="CTAO")
agent.name ="CTA Observatory"
agent.type = "Organization"
session.add(agent)

# For each activity
for cta_activity in p.provenance[:-1]:
    add_activity(session, cta_activity)
    
    # For each input file
    for cta_input in cta_activity['input']:
        
        # Get the id of the file
        filename_uuid = get_file_id(cta_input['url'])
            
        # If Entity does not exist in the database, add it - current_input_file.entityDescription_id= ???
        if filename_uuid == "":
            current_input_file = DatasetEntity(id=hfilename_uuid, classType = 'dataset', \
                                    name = logical_name, location = cta_input['url'])
            session.add(current_input_file)
            
            # Attribution to the agent - wAT.role = ?
            wAT = WasAttributedTo(entity = filename_uuid, agent = "CTAO")
            session.add(wAT)
            
        # Add the Used relationship
        used1 = Used(role = cta_input['role'], activity_id = cta_activity['activity_uuid'], entity_id = filename_uuid) # incremental id
        session.add(used1)
    
    # For each output file
    for cta_output in cta_activity['output']:
        
        # Computation of the hash of the file to determine the id of it
        filename_uuid = set_file_id(cta_output['url'])
            
        # If Entity already exists in the database, raise an Exception or a error message - #current_output_file.entityDescription_id= ???
        if session.query(exists().where(DatasetEntity.id==filename_uuid)):
            print ("ERROR")
        else:
            current_output_file = DatasetEntity(id=filename_uuid, classType = 'dataset', name = cta_output['url'].split('/')[-1],\
                                               location = cta_output['url'])
            session.add(current_output_file)
            
            # Attribution to the agent - wAT.role = ?
            wAT = WasAttributedTo(entity = filename_uuid, agent = "CTAO")
            session.add(wAT)
            
        # Add the wasgeneratedBy relationship - incremental id
        wGB1 = WasGeneratedBy(role = cta_output['role'], activity_id = cta_activity['activity_uuid'],\
                             entity_id = filename_uuid) 
        session.add(wGB1)
        
    # Add the status as an output ValueEntity
    current_output_value = ValueEntity(id=cta_activity['activity_uuid']+'_status')
    current_output_value.name = 'status'
    current_output_value.classType = 'value'
    current_output_value.valueXX = cta_activity['status']
    current_output_value.entityDescription_id = 'status'
    #current_output_value.location
    #current_output_value.entityDescription_id= ???
    session.add(current_output_value)
    
        
session.commit()

cle existante
ERROR
cle existante
ERROR
2019-04-02 09:40:58,819 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-04-02 09:40:58,822 INFO sqlalchemy.engine.base.Engine INSERT INTO "wasGeneratedBy" (role, activity_id, entity_id) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.base.Engine:INSERT INTO "wasGeneratedBy" (role, activity_id, entity_id) VALUES (?, ?, ?)


2019-04-02 09:40:58,824 INFO sqlalchemy.engine.base.Engine ('dl1.tel.evt.muon', 'ed7e27d4-a0a7-43a2-97a1-511348dd37bd', '4b19eb8d-21a9-5351-b4cc-09d0790f594e')


...

INFO:sqlalchemy.engine.base.Engine:INSERT INTO agents (id, name, type, email, affiliation, phone, address) VALUES (?, ?, ?, ?, ?, ?, ?)


2019-04-02 09:40:58,872 INFO sqlalchemy.engine.base.Engine ('CTAO', 'CTA Observatory', 'Organization', None, None, None, None)


INFO:sqlalchemy.engine.base.Engine:('CTAO', 'CTA Observatory', 'Organization', None, None, None, None)


2019-04-02 09:40:58,874 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


In [16]:
# Put the results in a file for the Provenance RFC
with open("muons_provRFC.txt", "w") as prov:
    prov.write("Provenance working example - CTA ctapipe-display-muons\n")
    prov.write("datamodel version 1.2 / preparation for PR-version 2 January 2019. MS.\n")
    prov.write("=========================================================================\n")
    prov.write("Remarks\n")
    prov.write("- ActivityDescription id = activity_name + '_' + ctapipe version\n")
    prov.write("- Activity id = uuid returned from ctapipe\n")  
    prov.write("- Entity id = hash (file)\n")
    prov.write("- Link between Entity and EntityDescription not defined. Via role?\n")
    prov.write("- Used and WasGeneratedBy Ids = activity id + '_' + 'entity id or incremental?\n")
    prov.write("\n")
    prov.write("\n")
    prov.write("=========================================================================\n")
    prov.write("\n")

    # Opérations sur le fichier
    for classname in [Activity, Entity, Used, WasGeneratedBy, \
                      DatasetEntity, ValueEntity, Parameter,\
                      ActivityDescription, EntityDescription, UsageDescription, GenerationDescription, \
                      DatasetDescription, ValueDescription, ParameterDescription,\
                      Agent, WasAttributedTo, WasAssociatedWith]:
        for instance in session.query(classname).order_by(classname.id):
            prov.write("%s\n" %instance)
prov.close()

2019-04-02 09:41:02,612 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)


2019-04-02 09:41:02,615 INFO sqlalchemy.engine.base.Engine SELECT activities.id AS activities_id, activities.name AS activities_name, activities."startTime" AS "activities_startTime", activities."endTime" AS "activities_endTime", activities.comment AS activities_comment, activities."activityDescription_id" AS "activities_activityDescription_id" 
FROM activities ORDER BY activities.id


INFO:sqlalchemy.engine.base.Engine:SELECT activities.id AS activities_id, activities.name AS activities_name, activities."startTime" AS "activities_startTime", activities."endTime" AS "activities_endTime", activities.comment AS activities_comment, activities."activityDescription_id" AS "activities_activityDescription_id" 
FROM activities ORDER BY activities.id


2019-04-02 09:41:02,617 INFO sqlalchemy.engine.base.Engine ()


...

INFO:sqlalchemy.engine.base.Engine:()


2019-04-02 09:41:02,729 INFO sqlalchemy.engine.base.Engine SELECT "wasAssociatedWith".id AS "wasAssociatedWith_id", "wasAssociatedWith".activity AS "wasAssociatedWith_activity", "wasAssociatedWith".agent AS "wasAssociatedWith_agent", "wasAssociatedWith".role AS "wasAssociatedWith_role" 
FROM "wasAssociatedWith" ORDER BY "wasAssociatedWith".id


INFO:sqlalchemy.engine.base.Engine:SELECT "wasAssociatedWith".id AS "wasAssociatedWith_id", "wasAssociatedWith".activity AS "wasAssociatedWith_activity", "wasAssociatedWith".agent AS "wasAssociatedWith_agent", "wasAssociatedWith".role AS "wasAssociatedWith_role" 
FROM "wasAssociatedWith" ORDER BY "wasAssociatedWith".id


2019-04-02 09:41:02,730 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [17]:
# Display the file contents
with open("muons_provRFC.txt", "r") as prov:
    for line in prov:
        print (line[:-1])
prov.close()

Provenance working example - CTA ctapipe-display-muons
datamodel version 1.2 / preparation for PR-version 2 January 2019. MS.
Remarks
- ActivityDescription id = activity_name + '_' + ctapipe version
- Activity id = uuid returned from ctapipe
- Entity id = hash (file)
- Link between Entity and EntityDescription not defined. Via role?
- Used and WasGeneratedBy Ids = activity id + '_' + 'entity id or incremental?



ValueEntity.id=c3b6e97b-b75e-4c4b-85eb-5d66609c0b0c_status
ValueEntity.classType=value
ValueEntity.name=status
ValueEntity.location=None
ValueEntity.generatedAtTime=None
ValueEntity.invalidatedAtTime=None
ValueEntity.comment=None
ValueEntity.entityDescription_id=status

ValueEntity.id=ed7e27d4-a0a7-43a2-97a1-511348dd37bd_status
ValueEntity.classType=value
ValueEntity.name=status
ValueEntity.location=None
ValueEntity.generatedAtTime=None
ValueEntity.invalidatedAtTime=None
ValueEntity.comment=None
ValueEntity.entityDescription_id=status

Used.id=1
Used.role=dl0.sub.evt
Used.time

In [ ]:
session.close()